In [ ]:
from typing import Optional, Literal, Callable, Sequence, Any
from pathlib import Path
from functools import partial
from multiprocessing import Pool
from tqdm.notebook import tqdm
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import pydicom
from PIL import Image
import cv2
import openpyxl
cv2.setNumThreads(0)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import lightning.pytorch as pl
from torchmetrics.classification import (
    MultilabelAUROC, MultilabelAveragePrecision
)
import shutil 
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.utilities.types import OptimizerLRScheduler

from torchvision.models.resnet import resnet50

from moonshot.model import moonshot
from moonshot.image_processor import ImageProcessor
from moonshot.utils import eval_mode
from pathlib import Path
from typing import Callable, Sequence, Optional
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models.resnet import resnet50
from torchvision.datasets import ImageFolder
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pydicom
import numpy as np
import os
import cv2
import torch
from PIL import Image
from PIL import UnidentifiedImageError
DEVICE = torch.device('cuda')
NUM_WORKERS = 16  # поменяйте на кол-во доступных CPU

# укажите полный или относительный путь до файла с весами модели Moonshot
MOONSHOT_MODEL_FILEPATH = Path('D:/vision_model.pt')  # путь до модели, если она лежит в корне репозитория

# --- 1. Настройка устройства ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
VINDRCXR_DIRPATH = Path(r"D:/l")
image_paths = list(VINDRCXR_DIRPATH.rglob("*.dcm"))
file = MOONSHOT_MODEL_FILEPATH
class MyModule(nn.Module): 
      
    # Initialize the parameter 
    def __init__(self, num_inputs, num_outputs, hidden_size): 
        super(MyModule, self).__init__() 
        self.linear1 = nn.Linear(num_inputs, hidden_size) 
        self.linear2 = nn.Linear(hidden_size, num_outputs) 
      
    # Forward pass 
    def forward(self, input): 
        lin    = self.linear1(input) 
        output = nn.functional.relu(lin) 
        pred   = self.linear2(output) 
        return pred
model = moonshot(MOONSHOT_MODEL_FILEPATH) #
model.load_state_dict(torch.load('D:/vision_model.pt')) #мы 
# load single image or dataset


Using device: cpu


D:\GitHub\Hackaton-2024\moonshot\model.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_filepath))
C:\Users\adm\AppData\Local\Temp

<All keys matched successfully>

In [8]:
class DICOMDataset(Dataset):
    def __init__(self, image_paths: Sequence[Path], labels: Sequence[int], transform: Optional[Callable] = None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx: int):
        dicom_file = pydicom.dcmread(str(self.image_paths[idx]))
        img = dicom_file.pixel_array
        img = cv2.resize(img, (518, 518))  # Преобразуем размер изображения

        # Если изображение ч/б, добавим фиктивный канал
        if len(img.shape) == 2:
            img = np.expand_dims(img, axis=-1)
        
        # Преобразуем к формату albumentations
        img = np.repeat(img, 3, axis=-1).astype(np.uint8)  # Преобразуем в 3 канала
        
        if self.transform:
            img = self.transform(image=img)["image"]

        label = self.labels[idx]
        return img, label


# --- 3. Аугментации ---
transform = A.Compose([
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])
labels = [0 if "benign" in str(p).lower() else 1 for p in image_paths]
dataset = DICOMDataset(image_paths, labels, transform)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


def train_model(model, train_loader, epochs, lr=1e-4):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    model.to(device)
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(imgs).squeeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")

    print("Training completed.")

In [9]:
train_model(model, train_loader, epochs=2, lr=1e-4)



: 